<H1>ПРОЕКТ: Доставка продуктов на дом.</h1>

Вы работаете продуктовым аналитиком в компании по доставке продуктов на дом. <br>
Сервис доступен как в приложении на ios, так и на android. <br>
Вы настроили фронтовую аналитику в AppMetrica, и в конце квартала маркетинг-менеджер <br>
попросил вас проанализировать поведение пользователей, а также оценить эффективность каналов их привлечения. <br>
<br>
Вы выгрузили данные из AppMetrica за период с 1 января по 31 марта 2020, только по пользователям, <br>
зарегистрированным позднее 1 января 2020. <br><br><br>



<H3>Описание данных:</H3>

- date – дата совершения события
- event - событие
<ol><li>app_install – установка приложения</li><li>app_start – открыть приложения</li><li>register – зарегистрироваться </li><li>search – перейти на страницу поиска товаров (каталог)</li><li>open_item – открыть товар</li><li>choose_item – отправить товар в корзину</li><li>tap_basket – перейти в корзину</li><li>purchase – подтверждение покупки</li></ol><br>
- gender – пол пользователя
- os_name – платформа пользователя
- city – город пользователя
- device_id – идентификатор устройства пользователя
- urm_source – канал, с которого пришел пользователь
<ol><li>yandex-direct – Яндекс директ</li>
<li>google_ads – реклама в Google</li>
<li>vk_ads – реклама в ВК</li>
<li>instagram_ads – реклама в instagram</li>
<li>facebook_ads – реклама в facebook</li>
<li>referal – акция «приведи друга» </li>
<li>Если стоит ‘-’, то канал не определен или это скачивание приложения напрямую или посещение не с рекламы </li></ol><br>

- purchase_sum – стоимость покупки (при совершении события ‘purchase’)<br><br><br>




Обратите внимание на следующее:
- В выгрузке только уникальные действия пользователей за каждый день 
- Можно миновать стадию установки приложения, если оно было установлено ранее
- Можно миновать стадию регистрации, если пользователь был уже залогинен на момент сессии. Однако незарегистрированные пользователи не могут оформить покупку. 

### БИБЛИОТЕКИ

In [1]:
# Подключим библиотеку pandas и назовем её pd
import pandas as pd
# Подключим библиотеку numpy и назовем её np
import numpy as np

# Импорт библиотек проводника
import os

# Подключим библиотеку seaborn и назовем её sns
import seaborn as sns

import datetime
from datetime import timedelta
from datetime import datetime
import time

### ЗАГРУЗКА ДАННЫХ

In [2]:
# Чтобы узнать текущую дирректорию
os.getcwd()
# Чтобы узнать что лежит в текущей дирректории
os.listdir()
# - ~/ - относительный путь
# - ../ - путь с поднятием вверх из другой папки до корневой
# (подъём на уровень выше - домашняя директория ~)

# Запишем путь файла в переменную
path_df_ecomm = "/mnt/HC_Volume_18315164/home-jupyter/jupyter-o-mihalev/Product_Analytics/Lesson_3/KC_case_data.csv"""
# Cчитаем DF c помощью лямбда-функции
download_df_ecomm = lambda x:  pd.read_csv(x, sep=',', parse_dates=[0]) #парсим дату
# Присвоим переменную датафрейму
df_ecomm = download_df_ecomm(path_df_ecomm)
df_ecomm

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads
...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads


### ПРОВЕРКА ДАННЫХ

In [3]:
# Проверим размер DF
print("Размер исходного датафрейма:\n{}".format(df_ecomm.shape), "\n")

# Преобразуем колонки с датой
#df_ecomm["date"] = pd.to_datetime(df_ecomm["date"])

# Проверим типы данных столбцов DF
print("Форматы колонок тисходного датафрейма:\n{}".format(df_ecomm.dtypes), '\n')

# Проверим DF на пропуски
print("Число пропусков в колонках исходного датафрейма:\n{}".format(df_ecomm.isna().sum()), '\n')

Размер исходного датафрейма:
(2747968, 8) 

Форматы колонок тисходного датафрейма:
date            datetime64[ns]
event                   object
purchase_sum           float64
os_name                 object
device_id                int64
gender                  object
city                    object
utm_source              object
dtype: object 

Число пропусков в колонках исходного датафрейма:
date                  0
event                 0
purchase_sum    2606585
os_name               0
device_id             0
gender                0
city                  0
utm_source            0
dtype: int64 



<h2>4. Расчёт MAU февраля</h2>

**MAU** -- месячная аудитория -- кол-во уникальных пользователей, которые зашли в продукт в течение месяца

In [4]:
# Cоздадим новую колонку с месяцем
df_ecomm["date_month"] = df_ecomm.date
df_ecomm.date_month = df_ecomm.date_month.dt.month_name()

# Будем считать юника по device_id
# Отфильтруем по февралю
df_ecomm_mau = df_ecomm.query("date_month == 'February'").device_id.nunique()

print(f"MAU февраля (число уникальных юзеров за февраль) = {df_ecomm_mau} человек.")

MAU февраля (число уникальных юзеров за февраль) = 75032 человек.


<h2>5. Расчёт количества установок в январе</h2>

In [5]:
df_ecomm_Ninst = df_ecomm.query("date_month == 'January' & event == 'app_install'").device_id.nunique()
print(f"Число установок за январь = {df_ecomm_Ninst} штук.")

Число установок за январь = 80297 штук.


<h2>6. Конверсия из установкив покупку в течение 7 дней.</h2>

Присвоим пользователям когорты по дню установки приложения.<br>
Для какой когорты конверсия была наибольшей? (формат: дд.мм.гггг)

Примечание: считаем пользователя сконвертировавшимся, если с момента установки <br>
до совершения первой покупки прошло не более 7 дней.

#### 1) Создадим DF с датами, когда произошла установка приложения

In [6]:
#  Отфильтруем по установкам
#  Отсортируем по дате и № устройства
# Удалим дубликаты № устройства, кроме самого первого (на всякий случай)
df_ecomm_install = df_ecomm.query("event == 'app_install'")\
                                                                .sort_values(["date", 'device_id'], ascending=True)\
                                                                .rename(columns={"date":"date_install"})\
                                                                .drop_duplicates("device_id", keep='first')
df_ecomm_install

,date_install,event,purchase_sum,os_name,device_id,gender,city,utm_source,date_month
2515778,2020-01-01,app_install,NaN,ios,4378,male,Saint-Petersburg,google_ads,January
2515286,2020-01-01,app_install,NaN,android,4505,female,Moscow,yandex-direct,January
2515922,2020-01-01,app_install,NaN,ios,4520,female,Saint-Petersburg,instagram_ads,January
2517537,2020-01-01,app_install,NaN,android,5248,female,Saint-Petersburg,facebook_ads,January
2516068,2020-01-01,app_install,NaN,ios,5524,male,Saint-Petersburg,vk_ads,January
...,...,...,...,...,...,...,...,...,...
2669431,2020-03-31,app_install,NaN,android,35315632,female,Moscow,-,March
2668887,2020-03-31,app_install,NaN,android,35370030,female,Saint-Petersburg,instagram_ads,March
2668697,2020-03-31,app_install,NaN,ios,35380796,female,Moscow,referal,March
2669045,2020-03-31,app_install,NaN,android,35388218,female,Saint-Petersburg,yandex-direct,March


#### 2) Присвоим когортные группы DF c установками

In [7]:
# Узнаем количесво уникальных дат установки
n_date_install = df_ecomm_install.date_install.nunique()
print(f"Количество уникальных дат с установкой приложения = {n_date_install}")

# Уникальные даты установки
uniq_date_install = pd.DataFrame(df_ecomm_install.date_install.drop_duplicates())
uniq_date_install = uniq_date_install.reset_index(drop=True)

# Присваиваем уникальным датам установок номера когортных групп
uniq_date_install["group"] = range(1, n_date_install + 1)
uniq_date_install

Количество уникальных дат с установкой приложения = 91


,date_install,group
0,2020-01-01,1
1,2020-01-02,2
2,2020-01-03,3
3,2020-01-04,4
4,2020-01-05,5
...,...,...
86,2020-03-27,87
87,2020-03-28,88
88,2020-03-29,89
89,2020-03-30,90


In [8]:
# Добавляем группы в DF с установками
df_ecomm_install = df_ecomm_install.merge(uniq_date_install, on="date_install", how="left")
df_ecomm_install

,date_install,event,purchase_sum,os_name,device_id,gender,city,utm_source,date_month,group
0,2020-01-01,app_install,NaN,ios,4378,male,Saint-Petersburg,google_ads,January,1
1,2020-01-01,app_install,NaN,android,4505,female,Moscow,yandex-direct,January,1
2,2020-01-01,app_install,NaN,ios,4520,female,Saint-Petersburg,instagram_ads,January,1
3,2020-01-01,app_install,NaN,android,5248,female,Saint-Petersburg,facebook_ads,January,1
4,2020-01-01,app_install,NaN,ios,5524,male,Saint-Petersburg,vk_ads,January,1
...,...,...,...,...,...,...,...,...,...,...
154592,2020-03-31,app_install,NaN,android,35315632,female,Moscow,-,March,91
154593,2020-03-31,app_install,NaN,android,35370030,female,Saint-Petersburg,instagram_ads,March,91
154594,2020-03-31,app_install,NaN,ios,35380796,female,Moscow,referal,March,91
154595,2020-03-31,app_install,NaN,android,35388218,female,Saint-Petersburg,yandex-direct,March,91


In [9]:
# Считаем установки по группам
installs = df_ecomm_install.groupby("group", as_index=False)\
                                                            .agg({"date_install": "count"})\
                                                            .rename(columns={"date_install":"installs"})
installs

,group,installs
0,1,3579
1,2,3144
2,3,2402
3,4,1831
4,5,1671
...,...,...
86,87,1199
87,88,1091
88,89,1117
89,90,994


In [10]:
# Cоздадим таблицу сообветствия групп и id
device = df_ecomm_install[["device_id","group" ]]
device

,device_id,group
0,4378,1
1,4505,1
2,4520,1
3,5248,1
4,5524,1
...,...,...
154592,35315632,91
154593,35370030,91
154594,35380796,91
154595,35388218,91


#### 3) Создадим DF с датами, когда произошла первая покупка

In [11]:
#  Отфильтруем по покупкам
#  Отсортируем по дате и № устройства
# Удалим дубликаты № устройства, кроме самого первого (на всякий случай)
df_ecomm_purchases = df_ecomm.query("event == 'purchase'")\
                                                                .sort_values(["date", 'device_id'], ascending=True)\
                                                                .rename(columns={"date":"date_purchase"})\
                                                                .drop_duplicates("device_id", keep='first')

df_ecomm_purchases

,date_purchase,event,purchase_sum,os_name,device_id,gender,city,utm_source,date_month
7914,2020-01-01,purchase,628.5,ios,4378,male,Saint-Petersburg,google_ads,January
8303,2020-01-01,purchase,72.0,android,4505,female,Moscow,yandex-direct,January
8432,2020-01-01,purchase,351.0,ios,6486,female,Saint-Petersburg,vk_ads,January
8547,2020-01-01,purchase,2674.5,ios,8674,female,Saint-Petersburg,-,January
8080,2020-01-01,purchase,78.0,android,9968,female,Saint-Petersburg,facebook_ads,January
...,...,...,...,...,...,...,...,...,...
2507067,2020-03-31,purchase,141.0,android,34964615,female,Saint-Petersburg,instagram_ads,March
2506627,2020-03-31,purchase,613.0,ios,35057676,male,Moscow,instagram_ads,March
2506590,2020-03-31,purchase,257.5,android,35171283,female,Moscow,vk_ads,March
2507015,2020-03-31,purchase,351.5,android,35315632,female,Moscow,-,March


In [12]:
# Примерджим дату установки и группы по id
df_ecomm_purchases = df_ecomm_purchases.merge(device, on="device_id", how="left").merge(uniq_date_install, on="group", how="left")

df_ecomm_purchases

,date_purchase,event,purchase_sum,os_name,device_id,gender,city,utm_source,date_month,group,date_install
0,2020-01-01,purchase,628.5,ios,4378,male,Saint-Petersburg,google_ads,January,1.0,2020-01-01
1,2020-01-01,purchase,72.0,android,4505,female,Moscow,yandex-direct,January,1.0,2020-01-01
2,2020-01-01,purchase,351.0,ios,6486,female,Saint-Petersburg,vk_ads,January,1.0,2020-01-01
3,2020-01-01,purchase,2674.5,ios,8674,female,Saint-Petersburg,-,January,1.0,2020-01-01
4,2020-01-01,purchase,78.0,android,9968,female,Saint-Petersburg,facebook_ads,January,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...
70870,2020-03-31,purchase,141.0,android,34964615,female,Saint-Petersburg,instagram_ads,March,91.0,2020-03-31
70871,2020-03-31,purchase,613.0,ios,35057676,male,Moscow,instagram_ads,March,91.0,2020-03-31
70872,2020-03-31,purchase,257.5,android,35171283,female,Moscow,vk_ads,March,91.0,2020-03-31
70873,2020-03-31,purchase,351.5,android,35315632,female,Moscow,-,March,91.0,2020-03-31


In [13]:
# Удалим тех, кто покупал, но не устанавливал приложение
df_ecomm_purchases = df_ecomm_purchases\
                                                                                .dropna(subset=["group"])\
                                                                                .astype({"group":"int"})
df_ecomm_purchases

,date_purchase,event,purchase_sum,os_name,device_id,gender,city,utm_source,date_month,group,date_install
0,2020-01-01,purchase,628.5,ios,4378,male,Saint-Petersburg,google_ads,January,1,2020-01-01
1,2020-01-01,purchase,72.0,android,4505,female,Moscow,yandex-direct,January,1,2020-01-01
2,2020-01-01,purchase,351.0,ios,6486,female,Saint-Petersburg,vk_ads,January,1,2020-01-01
3,2020-01-01,purchase,2674.5,ios,8674,female,Saint-Petersburg,-,January,1,2020-01-01
5,2020-01-01,purchase,542.0,android,11188,female,Moscow,google_ads,January,1,2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...
70870,2020-03-31,purchase,141.0,android,34964615,female,Saint-Petersburg,instagram_ads,March,91,2020-03-31
70871,2020-03-31,purchase,613.0,ios,35057676,male,Moscow,instagram_ads,March,91,2020-03-31
70872,2020-03-31,purchase,257.5,android,35171283,female,Moscow,vk_ads,March,91,2020-03-31
70873,2020-03-31,purchase,351.5,android,35315632,female,Moscow,-,March,91,2020-03-31


In [14]:
# Узнаем разницу между первой покупкой и установкой
df_ecomm_purchases["date_delta"] = df_ecomm_purchases.date_purchase - df_ecomm_purchases.date_install

df_ecomm_purchases

,date_purchase,event,purchase_sum,os_name,device_id,gender,city,utm_source,date_month,group,date_install,date_delta
0,2020-01-01,purchase,628.5,ios,4378,male,Saint-Petersburg,google_ads,January,1,2020-01-01,0 days
1,2020-01-01,purchase,72.0,android,4505,female,Moscow,yandex-direct,January,1,2020-01-01,0 days
2,2020-01-01,purchase,351.0,ios,6486,female,Saint-Petersburg,vk_ads,January,1,2020-01-01,0 days
3,2020-01-01,purchase,2674.5,ios,8674,female,Saint-Petersburg,-,January,1,2020-01-01,0 days
5,2020-01-01,purchase,542.0,android,11188,female,Moscow,google_ads,January,1,2020-01-01,0 days
...,...,...,...,...,...,...,...,...,...,...,...,...
70870,2020-03-31,purchase,141.0,android,34964615,female,Saint-Petersburg,instagram_ads,March,91,2020-03-31,0 days
70871,2020-03-31,purchase,613.0,ios,35057676,male,Moscow,instagram_ads,March,91,2020-03-31,0 days
70872,2020-03-31,purchase,257.5,android,35171283,female,Moscow,vk_ads,March,91,2020-03-31,0 days
70873,2020-03-31,purchase,351.5,android,35315632,female,Moscow,-,March,91,2020-03-31,0 days


In [15]:
# Отфильтруем по дельте <= 7 дней
#  Посчитаем покупки по группам
purchases = df_ecomm_purchases\
                                              .query("date_delta <= '7 days'")\
                                             .groupby("group", as_index=False)\
                                             .agg({"date_month": "count"})\
                                             .rename(columns={"date_month":"purchases"})

purchases

,group,purchases
0,1,1408
1,2,1186
2,3,834
3,4,639
4,5,587
...,...,...
86,87,277
87,88,209
88,89,223
89,90,204


#### 4) Объединим покупки и установки, посчитаем конверсию в покупку от установки

In [16]:
df_ecomm_CR = uniq_date_install\
                                            .merge(installs, on="group", how="left")\
                                            .merge(purchases, on="group", how="left")\

df_ecomm_CR["CR"] = (df_ecomm_CR.purchases / df_ecomm_CR.installs).mul(100).round(1
                                                                                 )

df_ecomm_CR = df_ecomm_CR.sort_values('CR', ascending=False)
df_ecomm_CR

,date_install,group,installs,purchases,CR
0,2020-01-01,1,3579,1408,39.3
8,2020-01-09,9,1424,558,39.2
14,2020-01-15,15,4310,1650,38.3
13,2020-01-14,14,5173,1973,38.1
1,2020-01-02,2,3144,1186,37.7
...,...,...,...,...,...
77,2020-03-18,78,1171,235,20.1
88,2020-03-29,89,1117,223,20.0
81,2020-03-22,82,1261,251,19.9
87,2020-03-28,88,1091,209,19.2


<h2>8. С какого платного маркетингового канала пришло<br>больше всего новых пользователей? </h2>

In [17]:
df_ecomm\
                    .query("utm_source != '-'")\
                    .drop_duplicates("device_id", keep='first')\
                    .groupby("utm_source", as_index=False)\
                    .agg({'device_id':"count"})\
                    .sort_values("device_id", ascending=False)\
                    .rename(columns={"device_id":"count_device_id"})

,utm_source,count_device_id
5,yandex-direct,36089
1,google_ads,33111
4,vk_ads,29379
2,instagram_ads,26370
0,facebook_ads,20430
3,referal,12908


<h2>9. На каком этапе воронки отваливается б0льшая часть клиентов? </h2>

Посмотрите отдельно сценарии для зарегистрированных и для незарегистрированных пользователей.<br>
На каком шаге отваливается больше всего зарегистрированных пользователей?<br><br>

В ответ впишите один из шагов, конверсия из которого (в следующий шаг) самая низкая, в таком формате: «Поиск», «Добавление товара», «Переход в корзину», «Регистрация».<br><br>

Подсказка: 

- Для того, чтобы выделить группу тех, кому нужно и не нужно регистрироваться, добавьте колонку с датой регистрации. Если дата регистрации < даты совершения события, то пользователь уже зарегистрирован. 

In [18]:
df_ecomm.event.unique()

array(['app_start', 'choose_item', 'purchase', 'search', 'tap_basket',
       'app_install', 'register'], dtype=object)

#### 1) Объединим покупки и установки, посчитаем конверсию в покупку от установки

In [19]:
#  Добавим дату регистрации для зарегистрированных пользователей
df_ecomm_register = df_ecomm.query("event == 'register'")
df_ecomm_register = df_ecomm_register.assign(date_register = df_ecomm.date)
df_ecomm_register = df_ecomm_register[["device_id", "date_register"]]
df_ecomm = df_ecomm.merge(df_ecomm_register, on=["device_id"], how="left")

In [20]:
df_ecomm = df_ecomm.merge(device, on=["device_id"], how="left")

In [21]:
df_ecomm = df_ecomm\
                                        .dropna(subset=["group"])\
                                        .astype({"group":"int"})

df_ecomm

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,date_month,date_register,group
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,January,2020-01-07,1
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,January,2020-01-01,1
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,January,2020-01-01,1
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,January,NaT,1
5,2020-01-01,app_start,NaN,android,5148476,male,Saint-Petersburg,-,January,2020-01-24,1
...,...,...,...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads,March,2020-03-31,88
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-,March,2020-03-31,91
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads,March,2020-03-31,91
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads,March,2020-03-31,66


#### 2) Установки

In [22]:
app_installs  = df_ecomm.query("event == 'app_install'")\
                                                            .groupby("group", as_index=False)\
                                                             .agg({"device_id": "count"})\
                                                             .rename(columns={"device_id":"app_install"})
app_installs

,group,app_install
0,1,3579
1,2,3144
2,3,2402
3,4,1831
4,5,1671
...,...,...
86,87,1199
87,88,1091
88,89,1117
89,90,994


 #### 3) Открытия приложения V

In [23]:
app_starts = df_ecomm.query("event == 'app_start'")\
                                                            .groupby("group", as_index=False)\
                                                             .agg({"device_id": "count"})\
                                                             .rename(columns={"device_id":"app_starts"})
app_starts

,group,app_starts
0,1,71516
1,2,52852
2,3,30604
3,4,20020
4,5,16076
...,...,...
86,87,1370
87,88,1205
88,89,1222
89,90,1043


#### 5) Поиск V

In [24]:
search = df_ecomm.query("event == 'search'")\
                                                            .groupby("group", as_index=False)\
                                                             .agg({"device_id": "count"})\
                                                             .rename(columns={"device_id":"search"})
search

,group,search
0,1,67419
1,2,49919
2,3,28921
3,4,18876
4,5,15236
...,...,...
86,87,1292
87,88,1141
88,89,1152
89,90,991


#### 6) Добавление товара V

In [25]:
add_item = df_ecomm.query("event == 'choose_item'")\
                                                            .groupby("group", as_index=False)\
                                                             .agg({"device_id": "count"})\
                                                             .rename(columns={"device_id":"add_item"})
add_item

,group,add_item
0,1,50659
1,2,37310
2,3,21655
3,4,14174
4,5,11434
...,...,...
86,87,1003
87,88,854
88,89,915
89,90,734


#### 7) Переход в корзину V

In [26]:
basket = df_ecomm.query("event == 'tap_basket'")\
                                                            .groupby("group", as_index=False)\
                                                             .agg({"device_id": "count"})\
                                                             .rename(columns={"device_id":"basket"})
basket

,group,basket
0,1,34707
1,2,25537
2,3,14816
3,4,9663
4,5,7879
...,...,...
86,87,702
87,88,619
88,89,629
89,90,541


#### 8) Регистрация V

In [27]:
# Если дата регистрации < даты совершения события, то пользователь уже зарегистрирован.
df_ecomm["reg_flag"] = df_ecomm.date_register <= df_ecomm.date

registrations = df_ecomm.query("reg_flag == True & event == 'register'")\
                                                            .groupby("group", as_index=False)\
                                                             .agg({"device_id": "count"})\
                                                             .rename(columns={"device_id":"register"})
registrations

,group,register
0,1,2430
1,2,2089
2,3,1505
3,4,1089
4,5,991
...,...,...
86,87,324
87,88,250
88,89,260
89,90,240


In [28]:
conversions = search.merge(add_item, on="group", how='left')\
                                        .merge(basket, on="group", how='left')\
                                        .merge(registrations, on="group", how='left')\

In [29]:
conversions["CR_search-additem"] = (conversions.add_item / conversions.search).mul(100).round(2)
conversions["CR_additem-basket"] = (conversions.basket / conversions.add_item).mul(100).round(2)
conversions["CR_basket-register"] = (conversions.register / conversions.basket).mul(100).round(2)

In [30]:
conversions

,group,search,add_item,basket,register,CR_search-additem,CR_additem-basket,CR_basket-register
0,1,67419,50659,34707,2430,75.14,68.51,7.00
1,2,49919,37310,25537,2089,74.74,68.45,8.18
2,3,28921,21655,14816,1505,74.88,68.42,10.16
3,4,18876,14174,9663,1089,75.09,68.17,11.27
4,5,15236,11434,7879,991,75.05,68.91,12.58
...,...,...,...,...,...,...,...,...
86,87,1292,1003,702,324,77.63,69.99,46.15
87,88,1141,854,619,250,74.85,72.48,40.39
88,89,1152,915,629,260,79.43,68.74,41.34
89,90,991,734,541,240,74.07,73.71,44.36


<h2>10. Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку? </h2>

В ответ впишите один из каналов в таком формате: «Яндекс», «Гугл», «ВК», «Фейсбук», «Инстаграм», «Реферальная программа».

Подсказка: 

- CR = кол-во юзеров, впервые совершивших покупку/кол-во юзеров, впервые открывших приложение 

In [31]:
df_ecomm

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,date_month,date_register,group,reg_flag
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,January,2020-01-07,1,False
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,January,2020-01-01,1,True
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,January,2020-01-01,1,True
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,January,NaT,1,False
5,2020-01-01,app_start,NaN,android,5148476,male,Saint-Petersburg,-,January,2020-01-24,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads,March,2020-03-31,88,True
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-,March,2020-03-31,91,True
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads,March,2020-03-31,91,True
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads,March,2020-03-31,66,True


#### 1) DF впервые открывших приложение

In [38]:
df_first_start_users = df_ecomm\
                                                .query("event == 'app_start'")\
                                                .sort_values("date", ascending=True)\
                                                .drop_duplicates(subset="device_id", keep='first')
df_first_start_users

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,date_month,date_register,group,reg_flag
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,January,2020-01-07,1,False
2955,2020-01-01,app_start,NaN,android,26774527,female,Moscow,yandex-direct,January,2020-03-25,1,False
2956,2020-01-01,app_start,NaN,android,28303290,male,Saint-Petersburg,-,January,2020-01-06,1,False
2957,2020-01-01,app_start,NaN,android,28472833,male,Saint-Petersburg,-,January,NaT,1,False
2958,2020-01-01,app_start,NaN,android,31632652,female,Moscow,facebook_ads,January,NaT,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...
2499247,2020-03-31,app_start,NaN,ios,8382261,female,Moscow,vk_ads,March,NaT,91,False
2499251,2020-03-31,app_start,NaN,ios,21705895,male,Saint-Petersburg,yandex-direct,March,NaT,91,False
2499252,2020-03-31,app_start,NaN,ios,24329615,male,Moscow,-,March,2020-03-31,91,True
2499253,2020-03-31,app_start,NaN,android,32732749,female,Moscow,yandex-direct,March,NaT,91,False


In [40]:
# Найдем кол-во юзеров зашедших впервые в приложение, по каналу привлечения

In [45]:
df_CR_first_start = df_first_start_users\
                                                                .groupby("utm_source", as_index = False)\
                                                                .agg({"device_id" : "count"})\
                                                                .rename(columns={"device_id" : "device_id_first_start"})
df_CR_first_start

,utm_source,device_id_first_start
0,-,32460
1,facebook_ads,13916
2,google_ads,26286
3,instagram_ads,20096
4,referal,9282
5,vk_ads,23189
6,yandex-direct,29368


#### 2) DF cовершивших покупку юзеров

In [42]:
df_purchases_users = df_ecomm\
                                                            .query("event == 'purchase'")\
                                                            .sort_values("date", ascending=True)\
                                                            .drop_duplicates(subset="device_id", keep='first')
df_purchases_users

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,date_month,date_register,group,reg_flag
7691,2020-01-01,purchase,265.5,android,6081155,female,Moscow,-,January,2020-01-01,1,True
8295,2020-01-01,purchase,551.0,ios,2072659,male,Saint-Petersburg,vk_ads,January,2020-01-01,1,True
8296,2020-01-01,purchase,172.0,android,28223736,male,Moscow,-,January,2020-01-01,1,True
8297,2020-01-01,purchase,666.0,ios,6069820,male,Saint-Petersburg,yandex-direct,January,2020-01-01,1,True
8298,2020-01-01,purchase,275.5,ios,9181435,female,Moscow,vk_ads,January,2020-01-01,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2506839,2020-03-31,purchase,374.5,ios,7391800,male,Moscow,vk_ads,March,2020-03-31,91,True
2506841,2020-03-31,purchase,261.0,ios,20672628,female,Saint-Petersburg,-,March,2020-03-31,79,True
2506842,2020-03-31,purchase,670.0,ios,32122725,male,Saint-Petersburg,-,March,2020-03-31,91,True
2506845,2020-03-31,purchase,407.0,ios,23188703,female,Moscow,instagram_ads,March,2020-03-31,91,True


In [ ]:
# Найдем кол-во юзеров совершивших покупку, по каналу привлечения

In [44]:
df_CR_purchases_users = df_purchases_users\
                                                                .groupby("utm_source", as_index = False)\
                                                                .agg({"device_id" : "count"})\
                                                                .rename(columns={"device_id" : "device_id_purchases_users"})
df_CR_purchases_users

,utm_source,device_id_purchases_users
0,-,11993
1,facebook_ads,6113
2,google_ads,8698
3,instagram_ads,7780
4,referal,4360
5,vk_ads,9108
6,yandex-direct,9406


#### 3) Объединим DF-ы и косчитаем конверсию в первую покупку по каналу привлечения

In [53]:
df_CR_first_purchases = df_CR_first_start\
                                                                    .merge(df_CR_purchases_users, how="right", on="utm_source")

df_CR_first_purchases = df_CR_first_purchases\
                                                                    .assign(CR = (df_CR_first_purchases.device_id_purchases_users / df_CR_first_purchases.device_id_first_start)\
                                                                                                                                                                                                                                                .mul(100).round(2))\
                                                                    .sort_values("CR", ascending=False)
df_CR_first_purchases

,utm_source,device_id_first_start,device_id_purchases_users,CR
4,referal,9282,4360,46.97
1,facebook_ads,13916,6113,43.93
5,vk_ads,23189,9108,39.28
3,instagram_ads,20096,7780,38.71
0,-,32460,11993,36.95
2,google_ads,26286,8698,33.09
6,yandex-direct,29368,9406,32.03


<h2>11. Пользователи, пришедшие с какого канала, имеют медианный первый чек выше? <br>
учитываются только первые покупки пользователей) </h2>

В ответ впишите один из каналов в таком формате: «Яндекс», «Гугл», «ВК», «Фейсбук», «Инстаграм», «Реферальная программа».

In [54]:
df_ecomm

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,date_month,date_register,group,reg_flag
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,January,2020-01-07,1,False
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,January,2020-01-01,1,True
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,January,2020-01-01,1,True
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,January,NaT,1,False
5,2020-01-01,app_start,NaN,android,5148476,male,Saint-Petersburg,-,January,2020-01-24,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads,March,2020-03-31,88,True
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-,March,2020-03-31,91,True
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads,March,2020-03-31,91,True
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads,March,2020-03-31,66,True


In [64]:
df_ecomm\
                .query("event == 'purchase'")\
                .sort_values("date", ascending=True)\
                .drop_duplicates(subset="device_id", keep='first')\
                .groupby("utm_source", as_index= False)\
                .agg({"device_id":"count",
                            "purchase_sum":"median"})\
                .rename(columns={"purchase_sum" : "median_check"})\
                .sort_values("median_check", ascending=False)

,utm_source,device_id,median_check
0,-,11993,399.00
4,referal,4360,395.00
6,yandex-direct,9406,392.75
3,instagram_ads,7780,392.50
5,vk_ads,9108,392.50
2,google_ads,8698,391.50
1,facebook_ads,6113,389.50


<h2>12. Какой платный канал привлечения (среди рекламных) имеет самый высокий ROMI? </h2>

Данные по затратам на рекламу:

- Яндекс – 10 491 707 руб.
- Гугл – 10 534 878 руб.
- Фейсбук – 8 590 498 руб.
- Инстаграм – 8 561626 руб.
- ВК – 9 553 531руб.

Расходы на реферальную программу: <br>
- если пользователь приведет друга и последний совершит первую покупку, то оба получат по 100 рублей.

Какой платный канал привлечения (среди рекламных) имеет самый высокий ROMI? 

В ответ впишите один из каналов в таком формате: «Яндекс», «Гугл», «ВК», «Фейсбук», «Инстаграм».

In [70]:
df_ecomm.utm_source.unique()

array(['-', 'vk_ads', 'facebook_ads', 'google_ads', 'instagram_ads',
       'yandex-direct', 'referal'], dtype=object)

#### 1) Найдем SUM_CHECK  - прибыль с платящих пользователей

In [174]:
# Найдем суммарный чек на уникального платящего пользователя
df_sum_check = df_ecomm\
                                            .query("event == 'purchase'")\
                                            .sort_values("date", ascending=True)\
                                            .groupby("utm_source", as_index= False)\
                                            .agg({"device_id":"nunique",
                                                        "purchase_sum":"sum"})\
                                            .rename(columns={"purchase_sum" : "sum_check",
                                                                                    "device_id":"nuniq_id"})\
                                            .sort_values("sum_check", ascending=False)

# Подсчёт LTV
df_sum_check = df_sum_check\
                                                .assign(ltv = df_sum_check["sum_check"] / df_sum_check["nuniq_id"]).round(2)
df_sum_check

,utm_source,nuniq_id,sum_check,ltv
0,-,13804,17922799.5,1298.38
5,vk_ads,10010,13074029.0,1306.10
6,yandex-direct,10495,11841717.0,1128.32
3,instagram_ads,8720,11577023.0,1327.64
2,google_ads,9865,11078830.5,1123.04
1,facebook_ads,7227,9805985.5,1356.85
4,referal,4919,6810794.0,1384.59


In [178]:
# отберем строку "referal"  выявления числа реферальных уников, которые совершили покупки
referal_nuniq = df_sum_check.loc[df_sum_check.utm_source == "referal"].nuniq_id[4]
print("Число пользователей совершивших покупку, пришедших по рефералке = {0}".format(referal_nuniq))

Число пользователей совершивших покупку, пришедших по рефералке = 4919


#### 2) Зададим Series с маркетинговыми тратами

In [179]:
# Зададим Series с маркетинговыми тратами
df_marketing = pd.Series({'vk_ads':9553531,
                                                      'facebook_ads':8590498, 
                                                      'google_ads':10534878, 
                                                      'instagram_ads':8561626,
                                                      'yandex-direct':10491707, 
                                                      'referal':100*2*referal_nuniq, 
                                                        "-":0})
df_marketing = pd.DataFrame(df_marketing)\
                                                                                            .reset_index()\
                                                                                            .rename(columns={"index" : "utm_source",
                                                                                                                                    0:"marketing_expenses"})\
                                                                                            .astype({"marketing_expenses" : "float64"}) # Изменим тип на float
df_marketing

,utm_source,marketing_expenses
0,vk_ads,9553531.0
1,facebook_ads,8590498.0
2,google_ads,10534878.0
3,instagram_ads,8561626.0
4,yandex-direct,10491707.0
5,referal,983800.0
6,-,0.0


#### 3) Посчитаем ROMI для каждого канала привлечения

In [180]:
df_sum_check = df_sum_check.merge(df_marketing, on="utm_source", how="left")
df_sum_check

,utm_source,nuniq_id,sum_check,ltv,marketing_expenses
0,-,13804,17922799.5,1298.38,0.0
1,vk_ads,10010,13074029.0,1306.10,9553531.0
2,yandex-direct,10495,11841717.0,1128.32,10491707.0
3,instagram_ads,8720,11577023.0,1327.64,8561626.0
4,google_ads,9865,11078830.5,1123.04,10534878.0
5,facebook_ads,7227,9805985.5,1356.85,8590498.0
6,referal,4919,6810794.0,1384.59,983800.0


In [183]:
df_sum_check = df_sum_check\
                                                .assign(romi = (df_sum_check["sum_check"] - df_sum_check["marketing_expenses"]) / df_sum_check["marketing_expenses"])\
                                                .round(2)\
                                                .sort_values("romi", ascending=False)
df_sum_check

,utm_source,nuniq_id,sum_check,ltv,marketing_expenses,romi
0,-,13804,17922799.5,1298.38,0.0,inf
6,referal,4919,6810794.0,1384.59,983800.0,5.92
1,vk_ads,10010,13074029.0,1306.10,9553531.0,0.37
3,instagram_ads,8720,11577023.0,1327.64,8561626.0,0.35
5,facebook_ads,7227,9805985.5,1356.85,8590498.0,0.14
2,yandex-direct,10495,11841717.0,1128.32,10491707.0,0.13
4,google_ads,9865,11078830.5,1123.04,10534878.0,0.05


In [191]:
df_sum_check.query("index != 0 & index != 6")

,utm_source,nuniq_id,sum_check,ltv,marketing_expenses,romi
1,vk_ads,10010,13074029.0,1306.10,9553531.0,0.37
3,instagram_ads,8720,11577023.0,1327.64,8561626.0,0.35
5,facebook_ads,7227,9805985.5,1356.85,8590498.0,0.14
2,yandex-direct,10495,11841717.0,1128.32,10491707.0,0.13
4,google_ads,9865,11078830.5,1123.04,10534878.0,0.05
